В данном ноутбуке представлен baseline решения задачи прогноза продаж продуктов в магазинах Эквадора.                    
                    
**Ссылка на соревнование:**  https://www.kaggle.com/c/store-sales-time-series-forecasting/overview                  
                           
**Описание:**                   
В данных 54 магазина и 33 семейства продуктов.                     
Временной ряд тренировочных данных с 01.01.2013 по 15.08.2017.              
Временной ряд тестовых данных для submition составляет 16 дней после последней даты тренировочных данных: с 16.08.2017 по 31.08.2017.                   
                                 
Нужно составить прогноз продаж для каждого из семейств продуктов в каждом из магазинов.     
                             
В отдельном ноутбуке проведен Feature engineering и сформированы необходимые датасеты.
                                
**Используемые для решения датасеты**                   
1. Train содержит временные ряды магазинов (столбец store_nbr) и комбинации семейств продуктов(столбец family). Столбец sales показывает общий объем продаж семейства продуктов в конкретном магазине на указанную дату.                   
2. Test - данные на тестовый период для submition.                       
3. zero_prediction - семейства товаров, которые не продавались в определенных магазинах весь рассматриваемый период (мы предполагаем, что продажи по данным товарам будут нулевые, поскольку этих товаров нет в магазинах) 

In [ ]:
import numpy as np
import pandas as pd

# PACF - ACF
# ------------------------------------------------------
import statsmodels.api as sm

# DATA VISUALIZATION
# ------------------------------------------------------
import matplotlib.pyplot as plt
import datetime as dt

%matplotlib inline
from tqdm.auto import tqdm

# Import Data

In [ ]:
# DATA_DIR = 'DATA/'
DATA_DIR = '../input/store-sales-time-series-forecasting/'
# DATA_zero = 'DATA/'
DATA_zero = '../input/zero-prediction/'

In [ ]:
# Import
train = pd.read_csv(DATA_DIR + "train.csv")
test = pd.read_csv(DATA_DIR + "test.csv")
zero_prediction = pd.read_csv(DATA_zero + "zero_prediction.csv")

# Datetime
train["date"] = pd.to_datetime(train.date)
test["date"] = pd.to_datetime(test.date)

zero_prediction = zero_prediction.set_index(['store_nbr', 'family', 'date']).sort_index()

In [ ]:
train.info()

In [ ]:
train.sample(3)

# Метрики

In [ ]:
def compute_metrics(real, forecast):
    result = {}
    real=np.array(real)
    forecast=np.array(forecast)
    result['MSE'] = round(((real-forecast)**2).mean(),4)
    result['RMSE'] = round((((real-forecast)**2)**(1/2)).mean(),4)
    mape_list = []
    smap_list = []
    for i in range(len(real)):
        if real[i] == 0:
            mape_list.append(0)
            smap_list.append(0)
        else:
            mape_list.append(abs(real[i]-forecast[i])/real[i])
            smap_list.append(2.0 * np.mean(np.abs(forecast[i] - real[i]) / (np.abs(forecast[i]) + np.abs(real[i]))))
        
    result['MAPE'] = round(np.mean(mape_list),4)
    result['SMAP'] = round(np.mean(smap_list),4)                   
    #result['SMAP'] = round(2.0 * np.mean(np.abs(forecast - real) / (np.abs(forecast) + np.abs(real))),4)
    return pd.Series(result)

# Validation

In [ ]:
train_df = train[train.date<'2017-08-01'].copy()
val_df = train[train.date>='2017-08-01'].copy()

# Среднее за последние 7 дней

In [ ]:
for num, store in enumerate(tqdm(train_df.store_nbr.unique())):
    for numf, fam in enumerate(tqdm(train_df.family.unique())):
        try:
            a=train_df[(train_df.store_nbr == store) & (train_df.family == fam)].copy()
            a=a.sales
            val_df.loc[(val_df.store_nbr == store) & (val_df.family == fam),'moving_avg']=a.rolling(7).mean().iloc[-1]
            test.loc[(test.store_nbr == store) & (test.family == fam),'moving_avg']=a.rolling(7).mean().iloc[-1]
            #print('store:',store,'fam:',fam)
        except:
            continue

In [ ]:
compute_metrics(val_df.sales, val_df.moving_avg)

In [ ]:
nbr=1
family = 'SEAFOOD'
train_viz = train[train.date>'2017-01-01'].copy()
train_viz = train_viz[(train_viz.store_nbr == nbr) & (train_viz.family == family)]
train_viz = train_viz.set_index('date')
val_viz = val_df.loc[(val_df.store_nbr == nbr) & (val_df.family == family)]
val_viz = val_viz.set_index('date')
test_viz = test.loc[(test.store_nbr == nbr) & (test.family == family)]
test_viz = test_viz.set_index('date')

plt.rcParams['figure.figsize'] = (15, 9)
plt.figure()
train_viz.sales.plot()
val_viz.moving_avg.plot()
test_viz.moving_avg.plot()
plt.title('Продажи в магазине №'+str(nbr)+' семейство товаров: '+family)
plt.show()

### Submit for moving_average

In [ ]:
test = test.reset_index().set_index(['store_nbr', 'family', 'date']).sort_index()
for i in zero_prediction.index:
    test.loc[i,'moving_avg']=0

In [ ]:
test=test.reset_index()
y_submit = test[['id','moving_avg']].copy()
y_submit.columns = ['id', 'sales']
y_submit.to_csv('submission_moving_avg.csv', index=False)

In [ ]:
y_submit.tail()

**Результат на kaggle 1.90138**

# Этот год = Прошлый год

In [ ]:
min_date = val_df.date.min()
max_date = val_df.date.max()
#train_df = train_df.reset_index()
last_year = train_df[(train_df.date >= (min_date - dt.timedelta(days=365))) & (train_df.date <= (max_date - dt.timedelta(days=365)))].copy()
last_year['date'] = last_year.date + dt.timedelta(days=365)
last_year = last_year[['store_nbr', 'family', 'date','sales']]
last_year.columns = ['store_nbr', 'family', 'date','last_year']
val_df = pd.merge(val_df, last_year[['store_nbr', 'family', 'date','last_year']], on=['store_nbr', 'family', 'date'], how='left')

In [ ]:
compute_metrics(val_df.sales, val_df.last_year)

In [ ]:
min_date = test.date.min()
max_date = test.date.max()
#train_df = train_df.reset_index()
last_year = train_df[(train_df.date >= (min_date - dt.timedelta(days=365))) & (train_df.date <= (max_date - dt.timedelta(days=365)))].copy()
last_year['date'] = last_year.date + dt.timedelta(days=365)
last_year = last_year[['store_nbr', 'family', 'date','sales']]
last_year.columns = ['store_nbr', 'family', 'date','last_year']
test = pd.merge(test, last_year[['store_nbr', 'family', 'date','last_year']], on=['store_nbr', 'family', 'date'], how='left')

In [ ]:
nbr=1
family = 'SEAFOOD'
train_viz = train[train.date>'2017-01-01'].copy()
train_viz = train_viz[(train_viz.store_nbr == nbr) & (train_viz.family == family)]
train_viz = train_viz.set_index('date')
val_viz = val_df.loc[(val_df.store_nbr == nbr) & (val_df.family == family)]
val_viz = val_viz.set_index('date')
test_viz = test.loc[(test.store_nbr == nbr) & (test.family == family)]
test_viz = test_viz.set_index('date')

plt.rcParams['figure.figsize'] = (15, 9)
plt.figure()
train_viz.sales.plot()
val_viz.last_year.plot()
test_viz.last_year.plot()
plt.show()

# Submit for last_year

In [ ]:
test = test.set_index(['store_nbr', 'family', 'date']).sort_index()
for i in zero_prediction.index:
    test.loc[i,'last_year']=0

In [ ]:
test=test.reset_index()
y_ly = test[['id','last_year']].copy()
y_ly.columns = ['id', 'sales']
y_ly.to_csv('submission_last_year.csv', index=False)

In [ ]:
y_ly.tail()

**Результат на kaggle 1.16088**